# 1. Libraries

In [26]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, ElasticNet, ARDRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.base import clone
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from xgboost import XGBRegressor
import shap

# 2. Data Loading

In [2]:
train = pd.read_parquet("../data/train.parquet")
X = train.drop(['label'], axis=1)
y = train['label']

# 3. Helper Functions

## 3.1 Scorer

In [2]:
def plot_ytrue_vs_ypred(y_true, y_pred, title="Predicted vs Actual"):
    plt.figure(figsize=(6,6))
    plt.scatter(y_true, y_pred, alpha=0.6)
    plt.plot([y_true.min(), y_true.max()], 
             [y_true.min(), y_true.max()], 
             'r--', lw=2)
    plt.xlabel("Actual (y_true)")
    plt.ylabel("Predicted (y_pred)")
    plt.title(title)
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.show()
    return

def scorer(y_pred, y_true, label="", verbose=True, plot=False, return_metric="r2"):

    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    if verbose:
        print("Regression Report for: " + label)
        print(f"MAE: {mae:.4f}")
        print(f"MSE: {mse:.4f}")
        print(f"R^2: {r2:.4f}")

    if plot:
        plot_ytrue_vs_ypred(y_true, y_pred)

    if return_metric == "r2":
        return r2
    elif return_metric == "mae":
        return mae
    elif return_metric == "mse":
        return mse
    else:
        raise ValueError("return_metric must be one of ['r2', 'mae', 'mse']")

## 3.2 Purged Cross Validation

In [ ]:
def purged_cv(model, X, y, n_splits=6, verbose=True, purge=True, scorer=scorer, return_metric="r2"):
    
    n = len(X)
    fold_size = n // n_splits
    all_scores = []

    for i in range(n_splits):
        test_start = i * fold_size
        test_end = (i + 1) * fold_size if i < n_splits - 1 else n
        test_idx = np.arange(test_start, test_end)

        purge_idx = []
        if purge:
            if i > 0:
                purge_idx.extend(range((i - 1) * fold_size, test_start))
            if i < n_splits - 1:
                purge_idx.extend(range(test_end, min((i + 2) * fold_size, n)))

        train_idx = np.setdiff1d(np.arange(n), np.concatenate([test_idx, purge_idx]) if purge else test_idx)

        if len(train_idx) == 0:
            continue

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        est = clone(model)

        if isinstance(est, LinearRegression) or isinstance(est, Lasso) or isinstance(est, Ridge) or isinstance(est, ElasticNet) or isinstance(est, ARDRegression):
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            est.fit(X_train, y_train)

        elif isinstance(est, XGBRegressor):

            if len(X_test) > 0:
                est.fit(
                    X_train, y_train,
                    eval_set=[(X_test, y_test)],
                    verbose=False
                )
            else:
                est.fit(X_train, y_train)
        else:
            est.fit(X_train, y_train)


        y_pred = est.predict(X_test)

        if verbose:
            score_val = scorer(y_pred, y_test, label=f"Fold {i}", verbose=verbose, plot=False, return_metric=return_metric)
        
        if return_metric == "r2":
            score_val = r2_score(y_test, y_pred)
        elif return_metric == "mae":
            score_val = mean_absolute_error(y_test, y_pred)
        elif return_metric == "mse":
            score_val = mean_squared_error(y_test, y_pred)
        else:
            raise ValueError("return_metric must be one of ['r2', 'mae', 'mse']")


        all_scores.append(score_val)

    return np.mean(all_scores) if all_scores else None

def purged_cv_shap_topk(model, X, y, n_splits=6, k=10, purge=True, scorer=None, return_metric="r2", verbose=True):
    n = len(X)
    fold_size = n // n_splits
    all_scores = []
    top_features_per_fold = []

    for i in range(n_splits):
        test_start = i * fold_size
        test_end = (i + 1) * fold_size if i < n_splits - 1 else n
        test_idx = np.arange(test_start, test_end)

        purge_idx = []
        if purge:
            if i > 0:
                purge_idx.extend(range((i - 1) * fold_size, test_start))
            if i < n_splits - 1:
                purge_idx.extend(range(test_end, min((i + 2) * fold_size, n)))

        train_idx = np.setdiff1d(np.arange(n), np.concatenate([test_idx, purge_idx]) if purge else test_idx)
        if len(train_idx) == 0:
            continue

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        est = clone(model)

        if isinstance(est, LinearRegression) or isinstance(est, LassoCV) or isinstance(est, Ridge) or isinstance(est, ElasticNet):
            from sklearn.preprocessing import StandardScaler
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            est.fit(X_train, y_train)

        # XGBRegressor fit
        elif isinstance(est, XGBRegressor):
            if len(X_test) > 0:
                est.fit(
                    X_train, y_train,
                    eval_set=[(X_test, y_test)],
                    verbose=False
                )
            else:
                est.fit(X_train, y_train)

        else:
            est.fit(X_train, y_train)

        y_pred = est.predict(X_test)
        if verbose and scorer is not None:
            score_val = scorer(y_pred, y_test, label=f"Fold {i}", verbose=verbose, plot=False, return_metric=return_metric)
        else:
            from sklearn.metrics import r2_score
            score_val = r2_score(y_test, y_pred)
        all_scores.append(score_val)

        explainer = shap.Explainer(est, X_train)
        shap_values = explainer(X_train) 

        shap_mean_abs = np.abs(shap_values.values).mean(axis=0)
        feature_importance = pd.Series(shap_mean_abs, index=X_train.columns)
        top_features = feature_importance.sort_values(ascending=False).head(k).index.tolist()
        top_features_per_fold.append(top_features)

    mean_score = np.mean(all_scores) if all_scores else None
    return mean_score, top_features_per_fold

def purged_cv_predict_only(model, X, y, n_splits=6, verbose=True, purge=True, scorer=None, return_metric="r2", scaler=None):
    n = len(X)
    fold_size = n // n_splits
    all_scores = []

    for i in range(n_splits):
        test_start = i * fold_size
        test_end = (i + 1) * fold_size if i < n_splits - 1 else n
        test_idx = np.arange(test_start, test_end)

        purge_idx = []
        if purge:
            if i > 0:
                purge_idx.extend(range((i - 1) * fold_size, test_start))
            if i < n_splits - 1:
                purge_idx.extend(range(test_end, min((i + 2) * fold_size, n)))

        train_idx = np.setdiff1d(np.arange(n), np.concatenate([test_idx, purge_idx]) if purge else test_idx)

        X_test = X.iloc[test_idx]
        y_test = y.iloc[test_idx]

        if isinstance(model, (LinearRegression, Lasso)):
            if scaler is None:
                raise ValueError("A fitted scaler must be provided for linear models.")
            X_test_scaled = scaler.transform(X_test)
            y_pred = model.predict(X_test_scaled)
        else:
            y_pred = model.predict(X_test)

        if scorer:
            score_val = scorer(y_pred, y_test, label=f"Fold {i}", verbose=verbose, plot=False, return_metric=return_metric)
        else:
            score_val = r2_score(y_test, y_pred)

        all_scores.append(score_val)

        if verbose:
            print(f"Fold {i}/{n_splits} → test size: {len(X_test)}, score: {score_val:.4f}")

    return np.mean(all_scores) if all_scores else None


# 4. Full Dataset Benchmarking

## 4.1 Linear Regression

In [74]:
purged_cv(LinearRegression(), X, y, n_splits=6, purge=True, scorer=scorer)

Regression Report for: Fold 0
MAE: 1.0311
MSE: 2.3691
R^2: -1.2586
Regression Report for: Fold 1
MAE: 0.9403
MSE: 1.6502
R^2: -0.7209


KeyboardInterrupt: 

## 4.2 Xgboost

In [ ]:
params = {
    "objective": "reg:pseudohubererror",
    "learning_rate": 0.01,       
    "n_estimators": 2000,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "max_depth": 4,              
    "gamma": 1,                  
    "tree_method": "hist",
    "early_stopping_rounds": 100,
    "eval_metric": "rmse"
}

xgb_model = XGBRegressor(**params)
#purged_cv(xgb_model, X, y, n_splits=6, purge=True, scorer=scorer)

# 5. Initial Feature Selection

In [99]:
with open("spearman_corr_mat.pkl", "rb") as f:
    corr_mat = pickle.load(f)

In [161]:
corrs = X.corrwith(y, method='pearson')
top_20_corrs = corrs.abs().sort_values(ascending=False).head(20).index

In [119]:
score, top_feats = purged_cv_shap_topk(xgb_model, X, y, n_splits=6, k=20)

100%|===================| 262616/262945 [11:46<00:00]        

In [ ]:
all_lists = top_feats + [['bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume']] + [top_20_corrs]
union_feats = list(set().union(*all_lists))

print(f"Number of features in union: {len(union_feats)}")

Number of features in union: 102


In [165]:
def filter_features(X, y, features, corr_thresh=0.9, target_corr_thresh=1e-4):

    X_sub = X[features].copy()

    target_corrs = X_sub.corrwith(y, method='pearson').abs()
    filtered = target_corrs[target_corrs >= target_corr_thresh].index.tolist()
    X_sub = X_sub[filtered]

    corr_matrix = X_sub.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    to_drop = [column for column in upper.columns if any(upper[column] > corr_thresh)]
    filtered_features = [f for f in X_sub.columns if f not in to_drop]

    return filtered_features

filtered = filter_features(X, y, union_feats, 0.9, 1e-4)
len(filtered)

73

In [ ]:
def forward_selection(X, y, max_features=None, threshold=0.0005, n_splits=6):
    remaining_features = list(X.columns)
    selected_features = []
    scores = []
    best_score = -float('inf')
    
    while remaining_features:
        scores_candidates = {}
        
        for feature in remaining_features:
            current_features = selected_features + [feature]
            model = LinearRegression()
            score = purged_cv(model, X[current_features], y, n_splits=n_splits, verbose=False)
            scores_candidates[feature] = score
        
        best_candidate = max(scores_candidates, key=scores_candidates.get)
        candidate_score = scores_candidates[best_candidate]
        
        if candidate_score - best_score > threshold:
            selected_features.append(best_candidate)
            remaining_features.remove(best_candidate)
            scores.append(candidate_score)
            best_score = candidate_score
            print(f"Selected: {best_candidate}, CV score: {candidate_score:.4f}")
            
            if max_features and len(selected_features) >= max_features:
                break
        else:
            break
    
    return selected_features, scores

#selected_features, scores = forward_selection(X, y, max_features=20, n_splits=6)
#print("Final selected features:", selected_features)

In [6]:
#curr_feats = selected_features + ["bid_qty", "ask_qty", "buy_qty", "sell_qty", "volume"]
curr_feats = ['X752', 'X21', 'X271', 'X109', 'X759', 'X445', 'X331', 'X508', 'X203', 'X386', 'X462', 'X625', 'X191'] + ["bid_qty", "ask_qty", "buy_qty", "sell_qty", "volume"]

In [10]:
purged_cv(LinearRegression(), X[curr_feats], y)

Regression Report for: Fold 0
MAE: 0.6232
MSE: 1.0069
R^2: 0.0400
Regression Report for: Fold 1
MAE: 0.6218
MSE: 0.9304
R^2: 0.0297
Regression Report for: Fold 2
MAE: 0.5806
MSE: 0.9902
R^2: 0.0181
Regression Report for: Fold 3
MAE: 0.6146
MSE: 0.9894
R^2: 0.0252
Regression Report for: Fold 4
MAE: 0.6839
MSE: 0.9339
R^2: 0.0186
Regression Report for: Fold 5
MAE: 0.7000
MSE: 1.0985
R^2: 0.0249


np.float64(0.026076031633525565)

In [ ]:
purged_cv(Ridge(alpha=1), X[curr_feats], y)

Regression Report for: Fold 0
MAE: 0.6232
MSE: 1.0069
R^2: 0.0400
Regression Report for: Fold 1
MAE: 0.6218
MSE: 0.9304
R^2: 0.0297
Regression Report for: Fold 2
MAE: 0.5806
MSE: 0.9902
R^2: 0.0181
Regression Report for: Fold 3
MAE: 0.6146
MSE: 0.9894
R^2: 0.0252
Regression Report for: Fold 4
MAE: 0.6839
MSE: 0.9339
R^2: 0.0186
Regression Report for: Fold 5
MAE: 0.7000
MSE: 1.0985
R^2: 0.0249


np.float64(0.026076068201283942)

In [4]:
params = {
    "booster": "gbtree",           
    "tree_method": "hist",          
    "learning_rate": 0.007101297601522795,
    "max_depth": 3,
    "min_child_weight": 2,
    "subsample": 0.010008709882592429,
    "colsample_bytree": 0.45890808231133295,
    "gamma": 3.541694744811049,
    "reg_lambda": 40.91405116528808,
    "reg_alpha": 4.603129764500413,
    "n_estimators": 1000,
    "random_state": 42,
    "verbosity": 0
}

#purged_cv(XGBRegressor(**params), X[curr_feats], y)

# 6. Feature Engineering

## Feature Expansion Plan

### Step 1: Linear Combinations
- Generate only linear combinations of original features (e.g., sums, differences, weighted sums).  
- **Feature Selection:**  
  - Filter features based on correlation with the target (threshold, e.g., |corr| > 0.01).  
  - Optionally, remove highly collinear features (|corr| > 0.9).  

### Step 2: 2nd-Order Combinations
- Generate pairwise interactions (products, ratios, differences) **only from original features**.  
- **Feature Selection:**  
  - Apply univariate filter (correlation or mutual information with target).  
  - Optional: apply a wrapper method (stepwise forward selection or Lasso) to select top features.  

### Step 3: 3rd-Order Combinations
- Generate triplet interactions **only from original features**.  
- **Feature Selection:**  
  - Use univariate filter first.  
  - Then apply a multivariate feature selection method:  
    - Lasso regression (for linear effects), or  
    - Tree-based importance (e.g., XGBoost SHAP values) to pick top-k features.  

**Notes:**  
- At each stage, **do not feed expanded features from previous stages** into the next expansion; always start from original features.  
- Keep track of selected features at each stage to avoid exponential growth.  
- Always apply correlation thresholding after selection to reduce redundancy.

In [7]:
original_feats = curr_feats

## 6.1 Combinations

## 6.2 Correlation Reduction

In [ ]:
def polynomial_feature_selection(X, y, degree=3, target_threshold=0.05, redundancy_threshold=0.85):
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly.fit_transform(X)
    feature_names = poly.get_feature_names_out(X.columns)
    X_poly = pd.DataFrame(X_poly, columns=feature_names, index=X.index)
    
    target_corr = X_poly.corrwith(y).abs()
    keep = target_corr[target_corr >= target_threshold].index
    X_filtered = X_poly[keep]
    target_corr = target_corr[keep]  
    
    corr_matrix = X_filtered.corr().abs()
    upper = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    to_drop = set()
    
    for i, f1 in enumerate(corr_matrix.columns):
        for j, f2 in enumerate(corr_matrix.columns):
            if upper[i, j] and corr_matrix.iloc[i, j] > redundancy_threshold:
                if target_corr[f1] < target_corr[f2]:
                    to_drop.add(f1)
                else:
                    to_drop.add(f2)
    
    selected_features = [f for f in X_filtered.columns if f not in to_drop]
    return X_filtered[selected_features], selected_features


filtered_df, curr_feats = polynomial_feature_selection(X[original_feats], y, target_threshold=1e-4, redundancy_threshold=0.9)

In [ ]:
print(curr_feats)
filtered_df.to_csv()

['X752', 'X21', 'X271', 'X109', 'X759', 'X445', 'X331', 'X508', 'X203', 'X386', 'X462', 'X625', 'X191', 'bid_qty', 'ask_qty', 'sell_qty', 'X752^2', 'X752 X21', 'X752 X271', 'X752 X109', 'X752 X759', 'X752 X445', 'X752 X331', 'X752 X508', 'X752 X203', 'X752 X386', 'X752 X462', 'X752 X625', 'X752 X191', 'X752 bid_qty', 'X752 ask_qty', 'X752 buy_qty', 'X21^2', 'X21 X271', 'X21 X109', 'X21 X759', 'X21 X445', 'X21 X331', 'X21 X508', 'X21 X203', 'X21 X386', 'X21 X462', 'X21 X625', 'X21 X191', 'X21 bid_qty', 'X21 ask_qty', 'X21 sell_qty', 'X271^2', 'X271 X109', 'X271 X759', 'X271 X445', 'X271 X331', 'X271 X508', 'X271 X203', 'X271 X386', 'X271 X462', 'X271 X625', 'X271 X191', 'X271 bid_qty', 'X271 ask_qty', 'X271 volume', 'X109^2', 'X109 X759', 'X109 X445', 'X109 X331', 'X109 X508', 'X109 X203', 'X109 X386', 'X109 X462', 'X109 X625', 'X109 X191', 'X109 bid_qty', 'X109 ask_qty', 'X109 volume', 'X759^2', 'X759 X445', 'X759 X331', 'X759 X508', 'X759 X203', 'X759 X386', 'X759 X462', 'X759 X625', 

In [14]:
mean_score, top_k_feats = purged_cv_shap_topk(XGBRegressor(**params), filtered_df[curr_feats], y)

100%|===================| 350471/350588 [35:58<00:00]        

In [26]:
list_of_lists = top_k_feats
union_set = set().union(*list_of_lists)
union_set = union_set.union(original_feats)
union_list = list(union_set)
print(union_list)
print(mean_score)

['volume', 'X21 sell_qty', 'X752^2 X21', 'X508', 'X109', 'bid_qty', 'X752 X508^2', 'X462', 'X752 X271 buy_qty', 'X759', 'X21^3', 'X386', 'X271 X508 ask_qty', 'X386 X625^2', 'X752 X271 X445', 'X331', 'X752 X331 ask_qty', 'sell_qty', 'X271', 'X752 buy_qty volume', 'X445', 'X271 X203 X462', 'X191', 'X203', 'X331 X203 X625', 'ask_qty', 'X203 X386 volume', 'X752^2 X271', 'X759 X445 X508', 'X752^2 X625', 'X752 X625^2', 'X752 X759^2', 'X508 X386 X462', 'X21 X271^2', 'X271 X445 X203', 'X752 X203^2', 'X21 X445 X386', 'X445 X625^2', 'X21 X508^2', 'X752', 'X625', 'X752 X331 buy_qty', 'X271 X508', 'X271 X445 ask_qty', 'X445 X203 X386', 'X21', 'X752 X445 X386', 'X752 X759 X625', 'buy_qty', 'X109^2 X759', 'X271^2 X331', 'X21 X191^2', 'X625^2', 'X445^3', 'X752 X109^2', 'X445 X386']
0.002100587141845023


In [27]:
union_list = ['volume', 'X21 sell_qty', 'X752^2 X21', 'X508', 'X109', 'bid_qty', 'X752 X508^2', 'X462', 'X752 X271 buy_qty', 'X759', 'X21^3', 'X386', 'X271 X508 ask_qty', 'X386 X625^2', 'X752 X271 X445', 'X331', 'X752 X331 ask_qty', 'sell_qty', 'X271', 'X752 buy_qty volume', 'X445', 'X271 X203 X462', 'X191', 'X203', 'X331 X203 X625', 'ask_qty', 'X203 X386 volume', 'X752^2 X271', 'X759 X445 X508', 'X752^2 X625', 'X752 X625^2', 'X752 X759^2', 'X508 X386 X462', 'X21 X271^2', 'X271 X445 X203', 'X752 X203^2', 'X21 X445 X386', 'X445 X625^2', 'X21 X508^2', 'X752', 'X625', 'X752 X331 buy_qty', 'X271 X508', 'X271 X445 ask_qty', 'X445 X203 X386', 'X21', 'X752 X445 X386', 'X752 X759 X625', 'buy_qty', 'X109^2 X759', 'X271^2 X331', 'X21 X191^2', 'X625^2', 'X445^3', 'X752 X109^2', 'X445 X386']

In [20]:
forward_selection(X, y, max_features=40, threshold=0.0005, n_splits=6)

Selected: X752, CV score: 0.0043
Selected: X21, CV score: 0.0117
Selected: X445^3, CV score: 0.0142
Selected: X331, CV score: 0.0180
Selected: X759, CV score: 0.0199
Selected: X109, CV score: 0.0212
Selected: X271, CV score: 0.0226
Selected: X508, CV score: 0.0240
Selected: X203, CV score: 0.0249
Selected: X386, CV score: 0.0257
Selected: X462, CV score: 0.0264
Selected: X191, CV score: 0.0270
Selected: X625, CV score: 0.0279


(['X752',
  'X21',
  'X445^3',
  'X331',
  'X759',
  'X109',
  'X271',
  'X508',
  'X203',
  'X386',
  'X462',
  'X191',
  'X625'],
 [np.float64(0.004269965602407555),
  np.float64(0.01169028461903221),
  np.float64(0.014168241095493583),
  np.float64(0.0180442318518416),
  np.float64(0.019902084077476383),
  np.float64(0.02117309612288069),
  np.float64(0.02261888902935168),
  np.float64(0.023954848478813462),
  np.float64(0.024895834406202244),
  np.float64(0.02574105525646873),
  np.float64(0.026388425334610093),
  np.float64(0.026959353285005667),
  np.float64(0.027901281538071403)])